In [1]:
import numpy as np
import pandas as pd
import shutil

import os,sys
from datetime import datetime

pd.set_option('display.max_columns', None)

In [2]:
df_warehouse=pd.read_excel(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\WAREHOUSE_DATA_FILE.xlsx")

In [3]:
df_warehouse.to_csv("warehouse_data.csv",index=False)

In [4]:
warehouse_Data = pd.read_csv("C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_data.csv")

In [5]:
assest_initiation_invent_sb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_SB.CSV")
assest_initiation_invent_nb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_NB.CSV")

In [6]:
assest_initiation_invent_sb.count()

MeterNumber                                  1023539
MFG_SERIAL_NUM                               1023539
UDC_ID                                       1023278
NETWORK_ID                                   1023278
InstallationNumber                            985085
Status                                       1023539
INSERT_TIME                                  1023539
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1023539
dtype: int64


# changes in dataset like :- UPPER, SPACE REMOVAL , DATATYPE

In [7]:
# Convert all columns to string data type

warehouse_Data = warehouse_Data.astype(str)
assest_initiation_invent_nb = assest_initiation_invent_nb.astype(str)
assest_initiation_invent_sb = assest_initiation_invent_sb.astype(str)



# change column name into upper string

warehouse_Data.columns = warehouse_Data.columns.str.upper()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.upper()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.upper()



# Remove leading and trailing whitespaces from all columns where dtype is 'O'

warehouse_Data = warehouse_Data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_nb = assest_initiation_invent_nb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_sb = assest_initiation_invent_sb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



# strip the spaces in column 

warehouse_Data.columns = warehouse_Data.columns.str.strip()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.strip()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.strip()


In [8]:
warehouse_Data.count()
# SHAPE OF DATA

print(f"DATASET SHAPE : {warehouse_Data.shape[0]}")

DATASET SHAPE : 3092


In [9]:
warehouse_Data.columns

Index(['PHASE', 'SUPPLIER', 'TYPE', 'DATE', 'REMARKS', 'WAREHOUSE_UTILITY',
       'METER NO.', 'METER MFG', 'IME', 'FINAL SIM', 'IMSI', 'IP'],
      dtype='object')

In [10]:
# concat the assest_initiation_invent_sb and assest_initiation_invent_nb

NB_SB_COMBINED_DATAFRAME = pd.concat([assest_initiation_invent_sb, assest_initiation_invent_nb], ignore_index=True)

In [11]:
# checking is null

NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME["NETWORK_ID"].isnull()].count()

METERNUMBER                                  0
MFG_SERIAL_NUM                               0
UDC_ID                                       0
NETWORK_ID                                   0
INSTALLATIONNUMBER                           0
STATUS                                       0
INSERT_TIME                                  0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    0
dtype: int64

In [12]:
# checking duplicate value

NB_SB_COMBINED_DATAFRAME.duplicated().sum()

0

In [13]:
# droping not used column in the NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop(['METERNUMBER', 'INSTALLATIONNUMBER','INSERT_TIME'], axis=1, inplace=True)

In [14]:
print(f" SHAPE OF NB_SB_COMBINED_DATAFRAME :- {NB_SB_COMBINED_DATAFRAME.shape[0]}")

 SHAPE OF NB_SB_COMBINED_DATAFRAME :- 1808299


In [15]:
# dop duplicate in  NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop_duplicates(subset=['MFG_SERIAL_NUM','UDC_ID',"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], keep='first', inplace=True, ignore_index=True)

In [16]:
print(f" SHAPE OF NB_SB_COMBINED_DATAFRAME :- {NB_SB_COMBINED_DATAFRAME.shape[0]}")

 SHAPE OF NB_SB_COMBINED_DATAFRAME :- 1713244


# CHANGES IN  NB_SB_COMBINED_DATAFRAME :- DATATYPE,UPPER,SPACES

In [17]:


# Convert all columns to string data type

NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.astype(str)


NB_SB_COMBINED_DATAFRAME.columns = NB_SB_COMBINED_DATAFRAME.columns.str.upper()



# Remove leading and trailing whitespaces from all columns where dtype is 'O'
NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)


# JOIN



In [18]:
#if ((warehouse_Data['METER NO.']== NB_SB_COMBINED_DATAFRAME['MFG_SERIAL_NUM']) and (warehouse_Data['WAREHOUSE_UTILITY']== NB_SB_COMBINED_DATAFRAME["DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"])):
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'WAREHOUSE_UTILITY'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['MFG_SERIAL_NUM'].isnull()]
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_DROPED_DATA = (LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.dropna(subset=['MFG_SERIAL_NUM','UDC_ID']))

if LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL.shape[0] > 0:
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL[warehouse_Data.columns]
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK=pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.'], right_on=['MFG_SERIAL_NUM'], how='left')
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.concat([LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_DROPED_DATA, LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK], ignore_index=True)
else:
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA


    


In [19]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['MFG_SERIAL_NUM'].isnull()]

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"


In [20]:
#NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME['MFG_SERIAL_NUM']=='GP4465021']

In [21]:


#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'WAREHOUSE_UTILITY'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')


In [22]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.count()

PHASE                                        3096
SUPPLIER                                     3096
TYPE                                         3096
DATE                                         3096
REMARKS                                      3096
WAREHOUSE_UTILITY                            3096
METER NO.                                    3096
METER MFG                                    3096
IME                                          3096
FINAL SIM                                    3096
IMSI                                         3096
IP                                           3096
MFG_SERIAL_NUM                               3096
UDC_ID                                       3096
NETWORK_ID                                   3096
STATUS                                       3096
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3096
dtype: int64

In [23]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.csv',index=False)

In [24]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['IMSI'], right_on=['UDC_ID'], how='left')

In [25]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

PHASE                                        3099
SUPPLIER                                     3099
TYPE                                         3099
DATE                                         3099
REMARKS                                      3099
WAREHOUSE_UTILITY                            3099
METER NO.                                    3099
METER MFG                                    3099
IME                                          3099
FINAL SIM                                    3099
IMSI                                         3099
IP                                           3099
MFG_SERIAL_NUM                               3094
UDC_ID                                       3094
NETWORK_ID                                   3094
STATUS                                       3094
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3094
dtype: int64

In [26]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].isnull()]

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
2419,1 Phase,L&T,Refurbished,2023-09-30,NOT MAP IN HES,NBPDCL,LT0143605,LNTLT0143605,861190056099168,8991856050706102326,405856706102326,2405:0203:0528:CDE3:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN
2420,1 Phase,L&T,Refurbished,2023-09-30,NOT MAP IN HES,NBPDCL,LT0166555,LNTLT0166555,861190057099704,8991856050706102325,405856706102325,2405:0203:0528:CE0F:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN
2421,1 Phase,L&T,Refurbished,2023-09-30,NOT MAP IN HES,NBPDCL,LT0168391,LNTLT0168391,861190056989004,8991856050706102138,405856706102138,2405:0203:0528:CDDA:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN
2422,1 Phase,L&T,Refurbished,2023-09-30,NOT MAP IN HES,NBPDCL,LT0171228,LNTLT0171228,861190057307818,8991856050706352212,405856706352212,2405:0203:0528:CDE1:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN
2423,1 Phase,L&T,Refurbished,2023-09-30,NOT MAP IN HES,NBPDCL,LT0181976,LNTLT0181976,861190056133736,8991856050706101941,405856706101941,2405:0203:0528:CDD2:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN


# duplicate row while we join with imsi 

In [27]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[~LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].isnull()].count()

In [28]:
# duplicated row ..  bcz in asset initiation file there are 2 installtion no.  with respect to 1 imsi no.

duplicate_check = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].duplicated(keep=False)]

duplicate_check.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv',index=False)
duplicate_check.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv",index=False)

In [29]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()


PHASE                                        3099
SUPPLIER                                     3099
TYPE                                         3099
DATE                                         3099
REMARKS                                      3099
WAREHOUSE_UTILITY                            3099
METER NO.                                    3099
METER MFG                                    3099
IME                                          3099
FINAL SIM                                    3099
IMSI                                         3099
IP                                           3099
MFG_SERIAL_NUM                               3094
UDC_ID                                       3094
NETWORK_ID                                   3094
STATUS                                       3094
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3094
dtype: int64

In [30]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.drop_duplicates(subset=['IMSI'],inplace=True)


In [31]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

PHASE                                        3092
SUPPLIER                                     3092
TYPE                                         3092
DATE                                         3092
REMARKS                                      3092
WAREHOUSE_UTILITY                            3092
METER NO.                                    3092
METER MFG                                    3092
IME                                          3092
FINAL SIM                                    3092
IMSI                                         3092
IP                                           3092
MFG_SERIAL_NUM                               3087
UDC_ID                                       3087
NETWORK_ID                                   3087
STATUS                                       3087
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3087
dtype: int64

In [32]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['METER NO.']=='LT0783643']

##### joining both the table "   LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA   "       and            " LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA "

In [33]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [34]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns={"UDC_ID": "METER_MATCH_UDC_ID", "NETWORK_ID": "METER_MATCH_IP", "STATUS": "METER_MATCH_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"METER_MATCH_UTILITY"},
    inplace=True
)



In [35]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column in modified column

In [36]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN

Index(['PHASE', 'SUPPLIER', 'TYPE', 'DATE', 'REMARKS', 'WAREHOUSE_UTILITY',
       'METER NO.', 'METER MFG', 'IME', 'FINAL SIM', 'IMSI', 'IP',
       'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID', 'METER_MATCH_IP',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY'],
      dtype='object')

In [37]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN]

In [38]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head()

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL
1,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0515112,LNTLT0515112,865575064337236,8991856040706015373,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,LT0515112,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL
2,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0575586,LNTLT0575586,865681062819025,8991856040706117561,405856706662583,2405:0203:0529:969C:0000:0000:0000:0010,LT0575586,405856706662583,2405:0203:0529:969C:0000:0000:0000:0010,Installed,NBPDCL
3,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0365316,LNTLT0365316,865575064792497,8991856050706401373,405856706401373,2405:0203:0525:CA8B:0000:0000:0000:0010,LT0365316,405856706401373,2405:0203:0525:CA8B:0000:0000:0000:0010,Installed,NBPDCL
4,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0804363,LNTLT0804363,865575064762433,8991856040706562204,405856707107213,2405:0203:052D:31F4:0000:0000:0000:0010,LT0804363,405856707107213,2405:0203:052D:31F4:0000:0000:0000:0010,Installed,NBPDCL


In [39]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED['METER_MATCH_UDC_ID']=='nan'].count()

PHASE                  7
SUPPLIER               7
TYPE                   7
DATE                   7
REMARKS                7
WAREHOUSE_UTILITY      7
METER NO.              7
METER MFG              7
IME                    7
FINAL SIM              7
IMSI                   7
IP                     7
MFG_SERIAL_NUM         7
METER_MATCH_UDC_ID     7
METER_MATCH_IP         7
METER_MATCH_STATUS     7
METER_MATCH_UTILITY    7
dtype: int64

In [40]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"METER_MATCH_UDC_ID","METER_MATCH_IP","METER_MATCH_STATUS","METER_MATCH_UTILITY"]]

In [41]:
# LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [42]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns = {"MFG_SERIAL_NUM":"IMSI_MATCHED_METER_NO","NETWORK_ID":"IMSI_MATCHED_IP","STATUS":"IMSI_MATCHED_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"IMSI_MATCHED_UTILITY"},
    inplace=True
)

In [43]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.head(1)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL


In [44]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN=LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN

Index(['PHASE', 'SUPPLIER', 'TYPE', 'DATE', 'REMARKS', 'WAREHOUSE_UTILITY',
       'METER NO.', 'METER MFG', 'IME', 'FINAL SIM', 'IMSI', 'IP',
       'IMSI_MATCHED_METER_NO', 'UDC_ID', 'IMSI_MATCHED_IP',
       'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY'],
      dtype='object')

In [45]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN]

In [46]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL


In [47]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"IMSI_MATCHED_METER_NO","IMSI_MATCHED_IP","IMSI_MATCHED_STATUS","IMSI_MATCHED_UTILITY"]]

### WORK ON LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED  and  LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED

In [48]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL
1,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0515112,LNTLT0515112,865575064337236,8991856040706015373,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,LT0515112,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL
2,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0575586,LNTLT0575586,865681062819025,8991856040706117561,405856706662583,2405:0203:0529:969C:0000:0000:0000:0010,LT0575586,405856706662583,2405:0203:0529:969C:0000:0000:0000:0010,Installed,NBPDCL


In [49]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL
1,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0515112,LNTLT0515112,865575064337236,8991856040706015373,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,LT0515112,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL
2,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0575586,LNTLT0575586,865681062819025,8991856040706117561,405856706662583,2405:0203:0529:969C:0000:0000:0000:0010,LT0575586,405856706662583,2405:0203:0529:969C:0000:0000:0000:0010,Installed,NBPDCL


In [50]:
#

# data correction

In [51]:
# Convert all columns to string data type

# change into str
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)


# ----------------------------------------------------------------------------------

# change into upper
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()


# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)




In [52]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL


# Merge IMSI  and METER  file

In [53]:
warehouse_meter_imsi_data= pd.DataFrame(columns=[warehouse_Data.columns])

In [54]:
warehouse_meter_imsi_data

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP


In [55]:
#  empty column add to the  warehouse_meter_imsi_data  which has already some empty column of warehouse data

In [56]:
warehouse_meter_imsi_data['METER_MATCH_UDC_ID'] = None
warehouse_meter_imsi_data['METER_MATCH_IP'] = None
warehouse_meter_imsi_data['METER_MATCH_STATUS'] = None
warehouse_meter_imsi_data['METER_MATCH_UTILITY'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_IP'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_STATUS'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_UTILITY'] = None


In [57]:
warehouse_meter_imsi_data

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [58]:
# this column are not same in all the dataframe so that i used the above method

In [59]:
"""
warehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'WAREHOUSE_UTILITY',
       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',
       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',
       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',
       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',
       'IMSI_MATCHED_UTILITY'])
"""

"\nwarehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'WAREHOUSE_UTILITY',\n       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',\n       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',\n       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',\n       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',\n       'IMSI_MATCHED_UTILITY'])\n"

In [60]:
warehouse_meter_imsi_data.head()

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [61]:
warehouse_meter_imsi_data = pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED, LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[['METER NO.', 'IMSI','IMSI_MATCHED_METER_NO','IMSI_MATCHED_IP','IMSI_MATCHED_STATUS','IMSI_MATCHED_UTILITY']], left_on=['METER NO.', 'IMSI'], right_on=['METER NO.', 'IMSI'], how='left')

In [62]:
warehouse_meter_imsi_data.head(2)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL,LT0555992,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL
1,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0515112,LNTLT0515112,865575064337236,8991856040706015373,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,LT0515112,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL,LT0515112,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL


In [63]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data = warehouse_meter_imsi_data.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data.columns = warehouse_meter_imsi_data.columns.str.upper()

# ----------------------------------------------------------------------------------


# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data = warehouse_meter_imsi_data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



# SIM SET CHECK STATUS

In [64]:
def fun_sim_status_check(row):
    if (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "SAME SET IN SAME UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN SAME UTILITY"
    elif (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "SAME SET IN DIFFERENT UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    else:
        return None

# Apply the function to create a new column
warehouse_meter_imsi_data['SIM SET CHECK STATUS'] = warehouse_meter_imsi_data.apply(fun_sim_status_check, axis=1)



# TRUE - FALSE COLUMN

In [65]:
def fun_check_true_false__METER_SIM(row):
    if row['IMSI']==row['METER_MATCH_UDC_ID']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_SIM"] =  warehouse_meter_imsi_data.apply(fun_check_true_false__METER_SIM, axis=1)

In [66]:
def fun_check_true_false_METER_ORG(row):
    if row['METER_MATCH_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_ORG, axis=1)

In [67]:
def fun_check_true_false_SIM_ORG(row):
    if row['IMSI_MATCHED_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["SIM_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_SIM_ORG, axis=1)

In [68]:
def fun_check_true_false_METER_IP(row):
    if row['METER_MATCH_IP']==row['IP']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_IP"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_IP, axis=1)

# SIM SET CHECK STATUS RECHECK COLUMN

In [69]:
def check_set_and_utility(row):
    if row['METER_SIM'] == True and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SAME SET IN SAME UTILITY"
    elif row['METER_SIM'] == True and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "SAME SET IN DIFFERENT UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "DIFFERENT SET IN SAME UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    elif row['METER_SIM']== False and row['METER_ORG']== True and row['SIM_ORG']== False and row['METER_IP']== False:
        return "REFURBISHED METER WITH NEW IMSI"
    else:
        return None


warehouse_meter_imsi_data["SIM SET CHECK STATUS RECHECK"] =  warehouse_meter_imsi_data.apply(check_set_and_utility, axis=1)


In [70]:
warehouse_meter_imsi_data.columns

Index(['PHASE', 'SUPPLIER', 'TYPE', 'DATE', 'REMARKS', 'WAREHOUSE_UTILITY',
       'METER NO.', 'METER MFG', 'IME', 'FINAL SIM', 'IMSI', 'IP',
       'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID', 'METER_MATCH_IP',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY', 'IMSI_MATCHED_METER_NO',
       'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY',
       'SIM SET CHECK STATUS', 'METER_SIM', 'METER_ORG', 'SIM_ORG', 'METER_IP',
       'SIM SET CHECK STATUS RECHECK'],
      dtype='object')

# Action taken column

In [71]:

def fun_action_taken_case_I(data_row):

    # case 1
    
    #if data_row['IMSI']== data_row['METER_MATCH_UDC_ID']:
        #return "SET IS IN INSTALLED STAGE"
    if (data_row['METER_MATCH_STATUS'] != "Installed")  & (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "IMSI IS IN INSTALLED STAGE"
    elif (data_row['METER_MATCH_STATUS'] == "Installed")  & (data_row['IMSI_MATCHED_STATUS'] != "Installed"):
        return "METER IS IN INSTALLED STAGE"
    
    # case 2 


    # set 1 and row['METER_IP'] == True
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True)) and (data_row['METER_IP'] == True):
        return "METER-SIM LINK FILE REQUIRED"



    # set 1 and row['METER_IP'] == False 
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == True)) and (data_row['METER_IP'] == False) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == False)) and (data_row['METER_IP'] == False) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    # set 2  "SAME SET IN DIFFERENT UTILITY"
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG'] == False) and (data_row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"

    #set 3  "DIFFERENT SET IN SAME UTILITY"
    
    elif (data_row['METER_SIM'] == False) and ((data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == True)) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
 
    # set 4  "DIFFERENT SET IN DIFFERENT UTILITY"
    elif (data_row['METER_SIM'] == False) and ((data_row['METER_ORG'] == False) and (data_row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    


    # new cases are added in this to handle imsi null
    
    elif ( data_row['IMSI_MATCHED_METER_NO'] =='nan') and ( data_row['IMSI_MATCHED_IP'] =='nan') and ( data_row['IMSI_MATCHED_STATUS'] =='nan') and (data_row['METER_ORG'] == True ):
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( data_row['IMSI_MATCHED_METER_NO'] =='nan' ) and ( data_row['IMSI_MATCHED_IP'] =='nan') and ( data_row['IMSI_MATCHED_STATUS'] =='nan') and (data_row['METER_ORG'] == False ):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( data_row['IMSI_MATCHED_METER_NO'] =='nan') and ( data_row['IMSI_MATCHED_IP'] =='nan') and ( data_row['IMSI_MATCHED_STATUS'] =='nan')  and ( data_row['METER_MATCH_UDC_ID'] =='nan') and ( data_row['METER_MATCH_IP'] =='nan') and ( data_row['METER_MATCH_STATUS'] =='nan'):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    
    

    
    else:
        return " NEED TO CHECK THIS FILE"
    
    
    
warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_case_I, axis=1)

In [73]:
warehouse_meter_imsi_data.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_meter_imsi_data.csv",index=False)

In [ ]:
# working on code as per requested by warehouse:  utility

# user option

In [ ]:
warehouse_utility_importation_demand =  input("ENTER THE WAREHOUSE ON DEMAND UTILITY:  ")
if warehouse_utility_importation_demand == "NBPDCL":
    print("working on NBPDCL data")
elif warehouse_utility_importation_demand == "SBPDCL":
    print(" working on SBPDCL data")
elif warehouse_utility_importation_demand == "NO DEMAND":
    print("meter is new , new imsi - code flow is same as before no interuption")
else:
    print("plz chose correct option")


working on NBPDCL data


In [ ]:
warehouse_meter_imsi_data['ACTION TAKEN'].unique()

array(['SET IS IN INSTALLED STAGE', 'IMSI IS IN INSTALLED STAGE',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE', ' NEED TO CHECK THIS FILE',
       'METER , SIM AND METER-SIM LINK FILE REQUIRED'], dtype=object)

In [ ]:
#actions = ["METER IS IN INSTALLED STAGE", "IMSI IS IN INSTALLED STAGE", "SET IS IN INSTALLED STAGE"]
#warehouse_meter_imsi_set_installed_Stage = warehouse_meter_imsi_data[warehouse_meter_imsi_data['ACTION TAKEN'].isin(actions)]



warehouse_meter_imsi_set_installed_Stage = warehouse_meter_imsi_data[(warehouse_meter_imsi_data['ACTION TAKEN'] == "METER IS IN INSTALLED STAGE") | 
                           (warehouse_meter_imsi_data['ACTION TAKEN'] == "IMSI IS IN INSTALLED STAGE") |
                           (warehouse_meter_imsi_data['ACTION TAKEN'] == "SET IS IN INSTALLED STAGE")]


In [ ]:
warehouse_meter_imsi_set_installed_Stage.head(1)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL,LT0555992,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE


In [ ]:
warehouse_meter_imsi_set_installed_Stage.columns

Index(['PHASE', 'SUPPLIER', 'TYPE', 'DATE', 'REMARKS', 'WAREHOUSE_UTILITY',
       'METER NO.', 'METER MFG', 'IME', 'FINAL SIM', 'IMSI', 'IP',
       'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID', 'METER_MATCH_IP',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY', 'IMSI_MATCHED_METER_NO',
       'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY',
       'SIM SET CHECK STATUS', 'METER_SIM', 'METER_ORG', 'SIM_ORG', 'METER_IP',
       'SIM SET CHECK STATUS RECHECK', 'ACTION TAKEN'],
      dtype='object')

In [ ]:
warehouse_meter_imsi_set_installed_Stage['IMSI_MATCHED_STATUS'].unique()

array(['Installed', 'Retired', 'Meter Shop', 'Refurbished'], dtype=object)

In [ ]:
def fun_meter_sim_set_installed_Stage(row):
    if (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS']=='Installed') and (row['IMSI_MATCHED_UTILITY']== 'NBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'BOTH INSTALLED IN SAME UTILITY'
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS']=='Installed') and (row['IMSI_MATCHED_UTILITY']== 'NBPDCL') and (row['IMSI_MATCHED_STATUS'] != 'Installed'):
        return 'METER IS IN INSTALLED STAGE'
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'NBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'IMSI IS IN INSTALLED STAGE'
    
    #  another cases
    
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'SIM AND METER-SIM LINK FILE REQUIRED'
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS']=='nan'):
        return 'SIM AND METER-SIM LINK FILE REQUIRED'
    
    elif (row['METER_MATCH_UTILITY'] == 'SBPDCL')  and  (row['METER_MATCH_STATUS'] =='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'METER , SIM AND METER-SIM LINK FILE REQUIRED'
    elif (row['METER_MATCH_UTILITY'] == 'SBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'METER , SIM AND METER-SIM LINK FILE REQUIRED'
  
    
warehouse_meter_imsi_set_installed_Stage["ACTION TAKEN NEW"] =  warehouse_meter_imsi_set_installed_Stage.apply(fun_meter_sim_set_installed_Stage, axis=1)

warehouse_meter_imsi_set_installed_Stage.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_meter_imsi_set_installed_Stage.csv",index=False)

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_2800\880093781.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  warehouse_meter_imsi_set_installed_Stage["ACTION TAKEN NEW"] =  warehouse_meter_imsi_set_installed_Stage.apply(fun_meter_sim_set_installed_Stage, axis=1)


In [ ]:
fda da  fdf 

SyntaxError: invalid syntax (1834545642.py, line 1)

In [ ]:
warehouse_meter_imsi_data[warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO']!='nan'].count()

PHASE                           1804
SUPPLIER                        1804
TYPE                            1804
DATE                            1804
REMARKS                         1804
WAREHOUSE_UTILITY               1804
METER NO.                       1804
METER MFG                       1804
IME                             1804
FINAL SIM                       1804
IMSI                            1804
IP                              1804
MFG_SERIAL_NUM                  1804
METER_MATCH_UDC_ID              1804
METER_MATCH_IP                  1804
METER_MATCH_STATUS              1804
METER_MATCH_UTILITY             1804
IMSI_MATCHED_METER_NO           1804
IMSI_MATCHED_IP                 1804
IMSI_MATCHED_STATUS             1804
IMSI_MATCHED_UTILITY            1804
SIM SET CHECK STATUS            1804
METER_SIM                       1804
METER_ORG                       1804
SIM_ORG                         1804
METER_IP                        1804
SIM SET CHECK STATUS RECHECK    1802
A

In [ ]:
warehouse_meter_imsi_data.head(2)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,4.05857E+14,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,True,None,METER IS IN INSTALLED STAGE
1,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0515112,LNTLT0515112,865575064337236,8991856040706015373,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,LT0515112,4.05857E+14,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,True,None,METER IS IN INSTALLED STAGE


In [ ]:
"""

def fun_action_taken_new_case(row):

    if (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == False) :
        return "METER , SIM AND METER SIM LINK REQUIRED"
        
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ) and (row['METER_MATCH_UDC_ID'].isnull()) &  (row['METER_MATCH_IP'].isnull()) &  (row['METER_MATCH_STATUS'].isnull()):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"


warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)

"""


'\n\ndef fun_action_taken_new_case(row):\n\n    if (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == False) :\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n        \n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ) and (row[\'METER_MATCH_UDC_ID\'].isnull()) &  (row[\'METER_MATCH_IP\'].isnull()) &  (row[\'METER_MATCH_STATUS\'].isnull()):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply

In [ ]:
"""

def fun_action_taken_new_case(row):
    if (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == False ):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS'])  and (not row['METER_MATCH_UDC_ID']) and (not row['METER_MATCH_IP']) and (not row['METER_MATCH_STATUS']):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"

warehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)


"""


'\n\ndef fun_action_taken_new_case(row):\n    if (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == False ):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\'])  and (not row[\'METER_MATCH_UDC_ID\']) and (not row[\'METER_MATCH_IP\']) and (not row[\'METER_MATCH_STATUS\']):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)\n\n\n'

# Finally done

In [ ]:
warehouse_meter_imsi_data.head(1)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,4.05857E+14,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,True,None,METER IS IN INSTALLED STAGE


In [ ]:

# Custom function to update meter numbers
def update_meter_number(meter_number):
    if meter_number.startswith('GP') or meter_number.startswith('GL'):
        return 'GOE' + meter_number
    elif meter_number.startswith('LT'):
        return 'LNT' + meter_number
    else:
        return meter_number

# Apply the custom function to each element in the 'METER NO.' column
warehouse_meter_imsi_data['NEW_METER_NUMBER'] = warehouse_meter_imsi_data['METER NO.'].apply(update_meter_number)




# FINAL FILE SAVE AT MULTIPLE LOCATION

In [ ]:
# Final file at desktop folder

warehouse_meter_imsi_data.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_Data_meter_sim_final.csv',index=False)


# Final file in code folder

warehouse_meter_imsi_data.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv",index=False)

In [ ]:
warehouse_meter_imsi_data_check=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv")

In [ ]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()

# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)

# NEED TO CHECK FILE

In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM = warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']==' NEED TO CHECK THIS FILE']

In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv",index=False)
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv',index=False)



In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV = pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv")

In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV.head()

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER


In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'].isnull()]

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER


In [ ]:
warehouse_meter_imsi_data_check.head(2)


,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,4.05857E+14,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,True,nan,METER IS IN INSTALLED STAGE,LNTLT0555992
1,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0515112,LNTLT0515112,865575064337236,8991856040706015373,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,LT0515112,4.05857E+14,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,True,nan,METER IS IN INSTALLED STAGE,LNTLT0515112


In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'] != "NEED TO CHECK THIS FILE"].count()

PHASE                           3094
SUPPLIER                        3094
TYPE                            3094
DATE                            3094
REMARKS                         3094
WAREHOUSE_UTILITY               3094
METER NO.                       3094
METER MFG                       3094
IME                             3094
FINAL SIM                       3094
IMSI                            3094
IP                              3094
MFG_SERIAL_NUM                  3094
METER_MATCH_UDC_ID              3094
METER_MATCH_IP                  3094
METER_MATCH_STATUS              3094
METER_MATCH_UTILITY             3094
IMSI_MATCHED_METER_NO           3094
IMSI_MATCHED_IP                 3094
IMSI_MATCHED_STATUS             3094
IMSI_MATCHED_UTILITY            3094
SIM SET CHECK STATUS            3094
METER_SIM                       3094
METER_ORG                       3094
SIM_ORG                         3094
METER_IP                        3094
SIM SET CHECK STATUS RECHECK    3094
A

In [ ]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['METER IS IN INSTALLED STAGE',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'IMSI IS IN INSTALLED STAGE', 'NEED TO CHECK THIS FILE',
       'SET IS IN INSTALLED STAGE',
       'METER , SIM AND METER-SIM LINK FILE REQUIRED'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check.head(2)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER
0,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0555992,LNTLT0555992,865681062851242,8991856040706094571,405856706639593,2405:0203:0529:1B24:0000:0000:0000:0010,LT0555992,4.05857E+14,2405:0203:0529:1B24:0000:0000:0000:0010,Installed,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,True,nan,METER IS IN INSTALLED STAGE,LNTLT0555992
1,1 Phase,L&T,Refurbished,2023-08-03,NOT MAP IN HES,NBPDCL,LT0515112,LNTLT0515112,865575064337236,8991856040706015373,405856706560395,2405:0203:0528:8D34:0000:0000:0000:0010,LT0515112,4.05857E+14,2405:0203:0528:8D34:0000:0000:0000:0010,Installed,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,True,nan,METER IS IN INSTALLED STAGE,LNTLT0515112


In [ ]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['METER IS IN INSTALLED STAGE',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'IMSI IS IN INSTALLED STAGE', 'NEED TO CHECK THIS FILE',
       'SET IS IN INSTALLED STAGE',
       'METER , SIM AND METER-SIM LINK FILE REQUIRED'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check['SIM SET CHECK STATUS RECHECK'].unique()

array(['nan', 'REFURBISHED METER WITH NEW IMSI',
       'SAME SET IN DIFFERENT UTILITY',
       'DIFFERENT SET IN DIFFERENT UTILITY'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['IMSI_MATCHED_METER_NO']=="nan"].count()

PHASE                           1292
SUPPLIER                        1292
TYPE                            1292
DATE                            1292
REMARKS                         1292
WAREHOUSE_UTILITY               1292
METER NO.                       1292
METER MFG                       1292
IME                             1292
FINAL SIM                       1292
IMSI                            1292
IP                              1292
MFG_SERIAL_NUM                  1292
METER_MATCH_UDC_ID              1292
METER_MATCH_IP                  1292
METER_MATCH_STATUS              1292
METER_MATCH_UTILITY             1292
IMSI_MATCHED_METER_NO           1292
IMSI_MATCHED_IP                 1292
IMSI_MATCHED_STATUS             1292
IMSI_MATCHED_UTILITY            1292
SIM SET CHECK STATUS            1292
METER_SIM                       1292
METER_ORG                       1292
SIM_ORG                         1292
METER_IP                        1292
SIM SET CHECK STATUS RECHECK    1292
A

In [ ]:
warehouse_meter_imsi_data_check['METER_MATCH_UDC_ID'].dtype

dtype('O')

In [ ]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['METER IS IN INSTALLED STAGE',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'IMSI IS IN INSTALLED STAGE', 'NEED TO CHECK THIS FILE',
       'SET IS IN INSTALLED STAGE',
       'METER , SIM AND METER-SIM LINK FILE REQUIRED'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE'].count()

PHASE                           2
SUPPLIER                        2
TYPE                            2
DATE                            2
REMARKS                         2
WAREHOUSE_UTILITY               2
METER NO.                       2
METER MFG                       2
IME                             2
FINAL SIM                       2
IMSI                            2
IP                              2
MFG_SERIAL_NUM                  2
METER_MATCH_UDC_ID              2
METER_MATCH_IP                  2
METER_MATCH_STATUS              2
METER_MATCH_UTILITY             2
IMSI_MATCHED_METER_NO           2
IMSI_MATCHED_IP                 2
IMSI_MATCHED_STATUS             2
IMSI_MATCHED_UTILITY            2
SIM SET CHECK STATUS            2
METER_SIM                       2
METER_ORG                       2
SIM_ORG                         2
METER_IP                        2
SIM SET CHECK STATUS RECHECK    2
ACTION TAKEN                    2
NEW_METER_NUMBER                2
dtype: int64

In [ ]:
#assest_initiation_invent_sb[assest_initiation_invent_sb['UDC_ID']=='404751250008001']

In [ ]:
#assest_initiation_invent_nb[assest_initiation_invent_nb['UDC_ID']=='404751250008001']

# creating meter-sim link file

In [ ]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)
# ----------------------------------------------------------------------------------
# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()
# ----------------------------------------------------------------------------------
# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



In [ ]:
#METER , SIM AND METER-SIM LINK FILE REQUIRED

#SIM AND METER-SIM LINK FILE REQUIRED

#METER-SIM LINK FILE REQUIRED


In [ ]:
# creating meter_sim_link_file

#CREATE_METER_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER,SIM AND METER-SIM LINK FILE REQUIRED') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER IS IN INSTALLED STAGE') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER , SIM AND METER-SIM LINK FILE REQUIRED')]

CREATE_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='SIM AND METER-SIM LINK FILE REQUIRED')]

CREATE_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER-SIM LINK FILE REQUIRED')]
                                                                          

In [ ]:
print(f" CREATE_METER_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : {CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")
print(f" CREATE_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : {CREATE_SIM_AND_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")
print(f" CREATE_METER_SIM_LINK_FILE - UNIQUE : {CREATE_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")


 CREATE_METER_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : ['METER , SIM AND METER-SIM LINK FILE REQUIRED']
 CREATE_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : ['SIM AND METER-SIM LINK FILE REQUIRED']
 CREATE_METER_SIM_LINK_FILE - UNIQUE : []


### Meter-sim link file for all 3 cases

In [ ]:
# this file is meter-sim link code

In [ ]:
def fun_meter_sim_link_file(row):

    return "{};EP2PMeter;{};{};{}".format(row['NEW_METER_NUMBER'], row['IMSI'],row['FINAL SIM'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
CREATE_METER_SIM_LINK_FILE['meter_sim_link_(M-S)_single_file_data'] = CREATE_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_35200\2655918992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_35200\2655918992.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND

In [ ]:
#EESL_BSNL_PO0000000402_00000_SIM_20231220_121002_SBPDCL
#404751250004001;8991759065212004001;fd00:0000:0800:CC19:0000:0000:0000:0001;;BSNLEDF;ACTIVE;SBPDCL

#  this code is for sim file for all the 2 cases

In [ ]:
def fun_sim_file(row):

    return "{};{};{};;BSNLEDF;ACTIVE;{}".format(row['IMSI'], row['FINAL SIM'],row['IP'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)
CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)


C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_35200\669809455.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)
C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_35200\669809455.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.appl

# THIS CODE IS FOR METER FILE FOR METER,SIM,METER-SIM LINK FILE

In [ ]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE'].unique()

array(['1 Phase'], dtype=object)

##### 1 phase and 3 phase

In [ ]:
# 1 phase and 3 phase meter file :   meter, sim , meter-sim link file 


CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE']=='1 Phase']
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE['NEW_METER_NUMBER'].str.startswith('GOE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE['NEW_METER_NUMBER'].str.startswith('LNT')]



CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE']=='3 Phase']
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE['NEW_METER_NUMBER'].str.startswith('GOE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE['NEW_METER_NUMBER'].str.startswith('LNT')]


In [ ]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.head(1)

,PHASE,SUPPLIER,TYPE,DATE,REMARKS,WAREHOUSE_UTILITY,METER NO.,METER MFG,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER,"meter_sim_link_(M,S & M-S)_single_file_data","sim_link_(M,S & M-S)_single_file_data"
1651,1 Phase,L&T,Refurbished,2023-09-23,NOT MAP IN HES,NBPDCL,LT0842818,LNTLT0842818,860953066028811,8991856040706592416,405856707137425,2405:0203:052D:C82B:0000:0000:0000:0010,LT0842818,405856707137425,2405:0203:052D:C82B:0000:0000:0000:0010,Retired,SBPDCL,LT0842818,2405:0203:052D:C82B:0000:0000:0000:0010,Retired,SBPDCL,SAME SET IN DIFFERENT UTILITY,True,False,False,True,SAME SET IN DIFFERENT UTILITY,"METER , SIM AND METER-SIM LINK FILE REQUIRED",LNTLT0842818,LNTLT0842818;EP2PMeter;405856707137425;8991856...,405856707137425;8991856040706592416;2405:0203:...


### 1 PHASE

In [ ]:
def fun_meter_1_PHASE_GENUS_file(row):

    return "{};{};EP2PMeter;6_D1;SKM145-s2;GOE;2019;2019;DLMS;;;G36B7.145090;;;3G;;;;;{};4059;TCP;32;;;;1A2B3C4D;0;1;0;GOE_1PH_WC;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS.apply(fun_meter_1_PHASE_GENUS_file, axis=1)

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_35200\1650417850.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS.apply(fun_meter_1_PHASE_GENUS_file, axis=1)


In [ ]:
def fun_meter_1_PHASE_LNT_file(row):

    return "{};{};EP2PMeter;6_D1;AURORA;LNT;2021;2021;DLMS;;;RN13.00 00;;8;3G;;;;;{};4059;TCP;32;;;;lnt1;2;1;0;EESL_1PH_WC_V1;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT.apply(fun_meter_1_PHASE_LNT_file, axis=1)

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_35200\4219330656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT.apply(fun_meter_1_PHASE_LNT_file, axis=1)


### 3 PHASE

In [ ]:
def fun_meter_3_PHASE_GENUS_file(row):

    return "{};{};EP2PMeter;7_D2;SKM345-s2;GOE;2019;2019;DLMS;;;GF6B7.325061;;;3G;;;;;{};4059;TCP;32;;;;1A2B3C4D;0;1;0;GOE_3PH_WC;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS.apply(fun_meter_3_PHASE_GENUS_file, axis=1)

In [ ]:
def fun_meter_3_PHASE_LNT_file(row):

    return "{};{};EP2PMeter;7_D2;REGOR;LNT;2021;2021;DLMS;;;3PBNC.82 01;;8;3G;;;;;{};4059;TCP;32;;;;lnt1;2;1;0;EESL_3PH_WC_V1;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT.apply(fun_meter_3_PHASE_LNT_file, axis=1)

#### concat file

In [ ]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL = pd.concat([CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS, CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT,CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS,CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT], ignore_index=True)

In [ ]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL.csv",index=False)
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)

In [ ]:
# assign :  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL  - to -   CREATE_METER_SIM_AND_METER_SIM_LINK_FILE

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL

In [ ]:
# SAVING METER-SIM LINK FILE TO OUTPUT LOCATION

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)
CREATE_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)
CREATE_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_METER_SIM_LINK_FILE.csv",index=False)

# CREATING A NEW DATA FRAME

#### WORKING ON METER-SIM LINK FILE 

In [ ]:
# 1 
# sim-meter link in meter,sim and meter-sim link file

meter_sim_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data']
if  not (meter_sim_link_M_S_AND_MS_LINK_file_df.empty):
    meter_sim_link_M_S_AND_MS_LINK_file_df.to_csv("S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_M_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_M_S_AND_MS_LINK.head(1)
else:
    print(" meter_sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


In [ ]:
# 2
# sim-meter link in sim and meter-sim link file

meter_sim_link_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_S_AND_MS_LINK_file_df=CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data']
if  not (meter_sim_link_S_AND_MS_LINK_file_df.empty):
    meter_sim_link_S_AND_MS_LINK_file_df.to_csv("S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_S_AND_MS_LINK.head(1)
else:
    print(" meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


In [ ]:
# 3
# sim-meter link in meter-sim link file

meter_sim_link_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_MS_LINK_file_df=CREATE_METER_SIM_LINK_FILE['meter_sim_link_(M-S)_single_file_data']
if  not (meter_sim_link_MS_LINK_file_df.empty):
    meter_sim_link_MS_LINK_file_df.to_csv("M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_MS_LINK.head(1)
else:
    print(" meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


 meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME


### working on sim link file

In [ ]:
# 1
# sim link file in meter,sim and meter-sim link file

sim_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
sim_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] 
if  not (sim_link_M_S_AND_MS_LINK_file_df.empty):
    sim_link_M_S_AND_MS_LINK_file_df.to_csv("M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None,index=False)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_M_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_M_AND_MS_LINK.head(1)
else:
    print(" sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


In [ ]:
# 2 
# sim link file in sim and meter-sim link file

sim_link_S_AND_MS_LINK_file_df = pd.DataFrame()
sim_link_S_AND_MS_LINK_file_df=CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data']
if  not (sim_link_S_AND_MS_LINK_file_df.empty):
    sim_link_S_AND_MS_LINK_file_df.to_csv("S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None,index=False)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_AND_MS_LINK.head(1)
else:
    print(" sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")

### working on meter-link file

In [ ]:
# 1 
# meter link in meter,sim and meter-sim link file

meter_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['Meter_link_(M,S & M-S)_single_file_data']
if  not (meter_link_M_S_AND_MS_LINK_file_df.empty):
    meter_link_M_S_AND_MS_LINK_file_df.to_csv("S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_EP2PMeter_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_EP2PMeter_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.head(1)
else:
    print(" meter_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")

In [ ]:
print(f"CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL: -   shape: - {CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL.shape[0]}")
print(f"CREATE_METER_SIM_AND_METER_SIM_LINK_FILE: -   shape: - {CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.shape[0]}")


CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL: -   shape: - 243
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE: -   shape: - 243


In [ ]:
new_df_check = EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.loc[2]
#print(f"len of new_df_check: {new_df_check}")
a=';'
word=""
for i in new_df_check:
    print(i)
    for j in i:
        word=word+j
        if a in j:
            #print(a)
            #print(f"word is : {word}")
            word=""
            #print("semicolon is here")
print(f"last word is : {word}")


GOEGL8922708;GL8922708;EP2PMeter;6_D1;SKM145-s2;GOE;2019;2019;DLMS;;;G36B7.145090;;;3G;;;;;860953062906622;4059;TCP;32;;;;1A2B3C4D;0;1;0;GOE_1PH_WC;1.0;GOEGL8922708;NBPDCL
last word is : NBPDCL


### funciton for del and create the  folder on particular path


In [ ]:
def folder_del_create(file_path):
    if os.path.exists(file_path):
        try:
            # Delete the folder and its contents
            shutil.rmtree(file_path)
            os.makedirs(file_path)
            print( "Folder at  deleted successfully.")
            print( "Folder at  created successfully.")
        except Exception as e:
            print(f"Error deleting folder: {e}")


#  METER,SIM AND METER-SIM LINK
M_S_AND_MS_MSLINK_FILE = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'
M_S_AND_MS_M_FILE = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER_FILE'
M_S_AND_MS_S_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\SIM_FILE'

#
S_AND_MSLINK_MSLINK_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE'
S_AND_MSLINK_S_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\SIM_FILE'

#METER-SIM LINK
MS_AND_MS_MSLINK_FILE=r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER-SIM LINK\METER-SIM_LINK_FILE'

folder_del_create(M_S_AND_MS_MSLINK_FILE)

Folder at  deleted successfully.
Folder at  created successfully.


In [ ]:
folder_del_create(M_S_AND_MS_MSLINK_FILE)
folder_del_create(M_S_AND_MS_M_FILE)
folder_del_create(M_S_AND_MS_S_FILE)

folder_del_create(S_AND_MSLINK_MSLINK_FILE)
folder_del_create(S_AND_MSLINK_S_FILE)


folder_del_create(MS_AND_MS_MSLINK_FILE)

Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.


# function for generate single file

In [ ]:
def generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix):
    data_frame_concat_data = pd.read_csv(df_file_path, header=None)

    # base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'

    num_folder = 0
    k = 100
    # new_folder_name = "EESL_GENUS_SINGLE_METER"

    new_folder_path = os.path.join(base_path, new_folder_name)
    os.makedirs(new_folder_path)

    # CURRENT_TIME_STAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Iterate through each row and save it as a separate file
    for index, row in data_frame_concat_data.iterrows():
        
        # Create a new DataFrame with the current row
        single_row_df = pd.DataFrame([row])
        #print(f"single_row_data : {single_row_df}")
        last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
        #print(f"last_word : {last_part_after_semicolon}")


        CURRENT_TIME_STAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
        #print(CURRENT_TIME_STAMP)
        
        #print(k)
        #print("-----------------")
        
        # Create the full path for the new folder

        if k > 995:
            k = 100
            final_new_folder_name = new_folder_name
            num_folder = num_folder + 1
            final_new_folder_name = new_folder_name + "_" + str(num_folder)
            new_folder_path = os.path.join(base_path, final_new_folder_name)
            os.makedirs(new_folder_path)
        else:
            file_name = generated_file_name_prefix + str(k) + generated_file_name_suffix + str(CURRENT_TIME_STAMP) + "_" + str(last_part_after_semicolon) + ".csv"
            #print(f"file name is {file_name}")
            file_path = os.path.join(new_folder_path, file_name)
            #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
            single_row_df.to_csv(file_path, index=False, header=False)
            k=k+1

        
    print("METER-SIM LINK FILE DONE")


In [ ]:
# METER-SIM LINK

# 1. EESL_GENUS_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_{utility}.csv


# SIM LINK FILE

# 2.  EESL_BSNL_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_{utility}.csv
# 3.  EESL_GENUS_PO0000000{k}_00000_SIM_{CURRENT_TIME_STAMP}_{utility}.csv

# METER FILE

# 4. EESL_GENUS_PO0000000{K}_00000_EP2PMeter_{CURRENT_TIME_STAMP}_{utility}.csv

In [ ]:

# SIM-Meter link file for all the three action taken...

# METER,SIM AND METER-SIM LINK  ----  METER-SIM_LINK_FILE
df_file_path =r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [ ]:

#change the df file path
#  SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE


df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [ ]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER-SIM LINK\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\PYTHON\\EDF_CODE_AUTOMATION_PYTHON\\METER_MATCHING_DATA\\M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv'

In [ ]:
# function for sim file 

In [ ]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\SIM_FILE'
new_folder_name = 'EESL_BSNL_SINGLE_METER'
generated_file_name_prefix = 'EESL_BSNL_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [ ]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\SIM_FILE'
new_folder_name = 'EESL_BSNL_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [ ]:
# meter file function


# METER,SIM AND METER-SIM LINK  ----  METER_LINK_FILE
df_file_path =r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_EP2PMeter_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_EP2PMeter_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [ ]:
#EESL_BSNL_PO0000000402_00000_SIM_20231220_121002_SBPDCL
#404751250004001;8991759065212004001;fd00:0000:0800:CC19:0000:0000:0000:0001;;BSNLEDF;ACTIVE;SBPDCL

In [ ]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\METER_IMPORTATION_ZIMP_FILE.zip'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")


KeyboardInterrupt: 

In [ ]:
  THE END LETS CHECK

SyntaxError: invalid syntax (4184175093.py, line 1)

# XML CODE

In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime
def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")
for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    semicolon_split = rows[0].split(';')
    CURRENT_TIME_STAMP = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    if semicolon_split[3] == '6_D1':
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "1", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
        ]
    else:
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "3", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
        ]

    # Start building the XML string
    xml_string = '<SDPSyncMessage xmlns="http://www.emeter.com/energyip/syncinterface/v8" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n'

    # Loop through the data and append to the XML string
    for row in csv_data:
        
        if row.get("verb"):
            xml_string += '    <header>\n'
        elif row.get("mRID"):
            xml_string += '    <payload>\n        <device>\n'
        elif row.get("name"):
            xml_string += '            <parameter>\n'
        else:
            xml_string += '    <payload>\n        <device>\n'
        
        for key, value in row.items():
            if key != "verb" and key != "mRID" and key != "name":
                if key != "value" and key != "startDate":
                    xml_string += f'            <{key}>{value}</{key}>\n'
                else:
                    xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "value" and key == "":
                xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "verb":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "mRID":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "name":
                xml_string += f'                <{key}>{value}</{key}>\n'



        if row.get("verb"):
            xml_string += '    </header>\n'
        elif row.get("name"):
            xml_string += '            </parameter>\n'
        #else:
            #xml_string += '        </device>\n    </payload>\n'

    # Close the root element
    xml_string += '         </device>\n     </payload>\n</SDPSyncMessage>'

    # Print the XML string for verification
    print(xml_string)
    # Save the XML string to a file
    with open('output.xml', 'w') as file:
        file.write(xml_string)

    # Print a message indicating successful save
    print("XML string saved to 'output.xml'")


    # Parse the XML string
    root = ET.fromstring(xml_string)
    break
    
    
        

In [ ]:
# XML CODE WITH OUTPUT SAVE IN DIFF FILE

In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime
import os

def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")


CURRENT_TIME = datetime.now().strftime('%Y%m%d_%H%M%S')

num_folder = 0
k = 100
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE'
new_folder_name = 'METER_XML_OUTPUT_M_S_&_MS_LINK'


new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)


for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    semicolon_split_for_file_name = rows[0].split(';')
    semicolon_split = rows[0].split(';')
            
    CURRENT_TIME_STAMP = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    if semicolon_split[3] == '6_D1':
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "1", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
            ]
    else:
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "3", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
            ]
    xml_string = '<SDPSyncMessage xmlns="http://www.emeter.com/energyip/syncinterface/v8" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n'

                # Loop through the data and append to the XML string
    for row in csv_data:
            
        if row.get("verb"):
            xml_string += '    <header>\n'
        elif row.get("mRID"):
            xml_string += '    <payload>\n        <device>\n'
        elif row.get("name"):
            xml_string += '            <parameter>\n'
        else:
            xml_string += '    <payload>\n        <device>\n'

        for key, value in row.items():
            if key != "verb" and key != "mRID" and key != "name":
                if key != "value" and key != "startDate":
                    xml_string += f'            <{key}>{value}</{key}>\n'
                else:
                    xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "value" and key == "":
                xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "verb":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "mRID":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "name":
                xml_string += f'                <{key}>{value}</{key}>\n'

        if row.get("verb"):
            xml_string += '    </header>\n'
        elif row.get("name"):
            xml_string += '            </parameter>\n'
            #else:
                #xml_string += '        </device>\n    </payload>\n'\
            
    xml_string += '         </device>\n     </payload>\n</SDPSyncMessage>'
            #root = ET.fromstring(xml_string)
    root = ET.ElementTree(ET.fromstring(xml_string))

    k=k+1 

    if k > 995:
            k = 100
            final_new_folder_name = new_folder_name
            num_folder = num_folder + 1
            final_new_folder_name = new_folder_name + "_" + str(num_folder)
            new_folder_path = os.path.join(base_path, final_new_folder_name)
            os.makedirs(new_folder_path)
    else:
        file_name = "Sync_Electric_Meter_" + semicolon_split_for_file_name[0] + "_"  + semicolon_split_for_file_name[-1] + "_" + str(CURRENT_TIME)  + "-" + str(k) + ".xml"
        print(file_name)
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        print(file_path)

        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        with open(file_path, 'w') as file:
            file.write(xml_string)
        
    
        

Sync_Electric_Meter_GOEGL8915918_NBPDCL_20240115_150121-101.xml
C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE\METER_XML_OUTPUT_M_S_&_MS_LINK\Sync_Electric_Meter_GOEGL8915918_NBPDCL_20240115_150121-101.xml
Sync_Electric_Meter_GOEGL8982967_NBPDCL_20240115_150121-102.xml
C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE\METER_XML_OUTPUT_M_S_&_MS_LINK\Sync_Electric_Meter_GOEGL8982967_NBPDCL_20240115_150121-102.xml
Sync_Electric_Meter_GOEGL8922708_NBPDCL_20240115_150121-103.xml
C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE\METER_XML_OUTPUT_M_S_&_MS_LINK\Sync_Electric_Meter_GOEGL8922708_NBPDCL_20240115_150121-103.xml
Sync_Electric_Meter_GOEGL8940402_NBPDCL_20240115_150121-104.xml
C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE\MET

# create multiple single file

In [ ]:
import pandas as pd

import os,sys
from datetime import datetime

k=100
# Load your file into a pandas DataFrame
data_frame_concat_data = pd.read_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\EESL_GENUS_PO0000000100_00000_SIM_Meter_Link_20240115_145223_NBPDCL.csv', header=None)  # Replace 'your_input_file.csv' with your actual file name

base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

num_folder = 0
new_folder_name = "EESL_GENUS_SINGLE_METER"

new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)

def get_current_time_stamp():
    return f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

CURRENT_TIME_STAMP = get_current_time_stamp()

# Iterate through each row and save it as a separate file
for index, row in data_frame_concat_data.iterrows():
    
    # Create a new DataFrame with the current row
    single_row_df = pd.DataFrame([row])
    #print(f"single_row_data : {single_row_df}")
    last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
    #print(f"last_word : {last_part_after_semicolon}")


    CURRENT_TIME_STAMP = get_current_time_stamp()
    #print(CURRENT_TIME_STAMP)
    
    print(k)
    print("-----------------")
    
    # Create the full path for the new folder

    if k > 995:
        k = 100
        num_folder = num_folder + 1
        new_folder_name = "EESL_GENUS_SINGLE_METER_" + str(num_folder)
        new_folder_path = os.path.join(base_path, new_folder_name)
        os.makedirs(new_folder_path)
    else:
        file_name = f'EESL_GENUS_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_{last_part_after_semicolon}.csv'
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        single_row_df.to_csv(file_path, index=False, header=False)
        k=k+1

    
print("METER-SIM LINK FILE DONE")

100
-----------------
101
-----------------
102
-----------------
103
-----------------
104
-----------------
105
-----------------
106
-----------------
107
-----------------
108
-----------------
109
-----------------
110
-----------------
111
-----------------
112
-----------------
113
-----------------
114
-----------------
115
-----------------
116
-----------------
117
-----------------
118
-----------------
119
-----------------
120
-----------------
121
-----------------
122
-----------------
123
-----------------
124
-----------------
125
-----------------
126
-----------------
127
-----------------
128
-----------------
129
-----------------
130
-----------------
131
-----------------
132
-----------------
133
-----------------
134
-----------------
135
-----------------
136
-----------------
137
-----------------
138
-----------------
139
-----------------
140
-----------------
141
-----------------
142
-----------------
143
-----------------
144
-----------------
145
------

# change into zip file
###### C:\Users\Ratan Kumar Jha\Desktop\EESL_GENUS_PO0000000\data_insert_single_file

In [ ]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")


Zip file created at: C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file


In [ ]:
def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")

CURRENT_TIME_STAMP = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    semicolon_split = rows[0].split(';')
    print(semicolon_split[3])
    break

6_D1


In [ ]:
# xml code

In [ ]:
for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    for i in rows:
        last_after_semicolon = rows[0].rsplit(';', 1)[-1]
        print(last_after_semicolon)
    break

SBPDCL


In [ ]:
for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    
    last_after_semicolon = rows[0].split(';')
    print(last_after_semicolon[0])
    break
    
    
    
    

GOEGP4465996


# sim link file

In [ ]:
import pandas as pd

import os,sys
from datetime import datetime

k=100
# Load your file into a pandas DataFrame
data_frame_concat_data = pd.read_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\EESL_GENUS_PO0000000100_00000_SIM_20240104_104844_NBPDCL.csv', header=None)  # Replace 'your_input_file.csv' with your actual file name

base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

num_folder = 0
new_folder_name = "EESL_BSNL_SINGLE_METER"

new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)

def get_current_time_stamp():
    return f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

CURRENT_TIME_STAMP = get_current_time_stamp()

# Iterate through each row and save it as a separate file
for index, row in data_frame_concat_data.iterrows():
    
    # Create a new DataFrame with the current row
    single_row_df = pd.DataFrame([row])
    #print(f"single_row_data : {single_row_df}")
    last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
    #print(f"last_word : {last_part_after_semicolon}")


    CURRENT_TIME_STAMP = get_current_time_stamp()
    #print(CURRENT_TIME_STAMP)
    
    print(k)
    print("-----------------")
    
    # Create the full path for the new folder

    if k > 995:
        k = 100
        num_folder = num_folder + 1
        new_folder_name = "EESL_BSNL_SINGLE_METER_" + str(num_folder)
        new_folder_path = os.path.join(base_path, new_folder_name)
        os.makedirs(new_folder_path)
    else:
        file_name = f'EESL_BSNL_PO0000000{k}_00000_SIM_{CURRENT_TIME_STAMP}_{last_part_after_semicolon}.csv'
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        single_row_df.to_csv(file_path, index=False, header=False)
        k=k+1

    
print("SIM LINK FILE DONE")

100
-----------------
101
-----------------
102
-----------------
103
-----------------
104
-----------------
105
-----------------
106
-----------------
107
-----------------
108
-----------------
109
-----------------
110
-----------------
111
-----------------
112
-----------------
113
-----------------
114
-----------------
115
-----------------
116
-----------------
117
-----------------
118
-----------------
119
-----------------
120
-----------------
121
-----------------
122
-----------------
123
-----------------
124
-----------------
125
-----------------
126
-----------------
127
-----------------
128
-----------------
129
-----------------
130
-----------------
131
-----------------
132
-----------------
133
-----------------
134
-----------------
135
-----------------
136
-----------------
137
-----------------
138
-----------------
139
-----------------
140
-----------------
141
-----------------
142
-----------------
143
-----------------
144
-----------------
145
------

In [ ]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")

Zip file created at: C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file


#### METER,SIM AND METER-SIM LINK
    - METER FILE PREPARE
    - SIM FILE PREPARE
    - SIM-METER LINK

#### SIM AND METER-SIM LINK FILE
    - SIM FILE
    - METER-SIM LINK

#### METER-SIM LINK
    - METER-SIM LINK

